**Goal**: Build an AI Agent that converts normal english text into Generation Alpha slang. Use LangGraph framework to accomplish the task.

**Inspiration**: Have you ever come across words like "Skibidi", "Rizz", "Sus" ?, if not, then you are ancient. Because that is how the newer generation alpha kids born between 2013 - 2024 are talking/messaging. No worries, this AI Agent got you covered.

*   Note: The AI Agent covers only prominent words and might not cover all the exisiting words.





**Install packages**

In [1]:
!pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install -qU "langchain[mistralai]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.0/264.0 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 20.6 MB/s eta 0:00:00


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


**Import packages**

In [3]:
import requests
import json
import os
from collections import Counter
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage, AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model
import gradio as gr
from langchain.schema import HumanMessage
from langchain.tools import tool


**Setup the Mistral AI API KEY**

In [4]:
os.environ["MISTRAL_API_KEY"] = MISTRAL_API_KEY

**Define a tool that looks and fetches the words that will be replaced by GenAlpha text.**

In [5]:
slang_dict = {
    "rizz": {
        "meaning": "Short for charisma; refers to someone's charm or smoothness, especially in flirting.",
        "example": "He's got mad rizz, everyone's swooning over him."
    },
    "Gyat": {
        "meaning": "An exclamation of admiration or surprise, often playful.",
        "example": "Gyat! That trick was insane!"
    },
    "Drip": {
        "meaning": "Refers to someone's stylish outfit or overall fashion sense.",
        "example": "Her drip is unmatched, look at those sneakers!"
    },
    "Skibidi": {
        "meaning": "Inspired by the Skibidi Toilet YouTube series, it's used to describe something chaotic, silly, or unexpectedly cool.",
        "example": "This dance challenge is so skibidi, it's hilarious!"
    },
    "Delulu": {
        "meaning": "Short for delusional, often used jokingly to describe someone with unrealistic ideas.",
        "example": "You think you can go viral overnight? That's delulu!"
    },
    "Sigma": {
        "meaning": "A term for someone independent, confident, or unconventional.",
        "example": "He doesn't care about trends, he's a total sigma."
    },
    "Bussin'": {
        "meaning": "Used to describe something exceptionally good, often food.",
        "example": "This pizza is bussin', can we order another one?"
    },
    "Mid": {
        "meaning": "Something mediocre or average.",
        "example": "That movie was so mid, I wouldn't watch it again."
    },
    "Ohio": {
        "meaning": "Meme-inspired term for something awkward, cringe-worthy, or odd.",
        "example": "That haircut? Total Ohio vibes."
    },
    "Cap": {
        "meaning": "A lie or exaggeration.",
        "example": "There's no way you saw Beyoncé, stop the cap."
    },
    "Slay": {
        "meaning": "To perform excellently or look amazing while doing something.",
        "example": "You slayed that outfit today, it's perfect!"
    },
    "Sheesh": {
        "meaning": "An exclamation of excitement or disbelief.",
        "example": "Sheesh! That car is incredible!"
    },
    "NPC": {
        "meaning": "Refers to someone who seems unoriginal or robotic, like a non-playable character in a video game.",
        "example": "He just repeats what everyone else says, total NPC vibes."
    },
    "Ong": {
        "meaning": "Short for \"on God,\" used to emphasize truth or sincerity.",
        "example": "This cake is amazing, ong!"
    },
    "Slaps": {
        "meaning": "Used to describe something excellent, often music or food.",
        "example": "This playlist slaps, who made it?"
    },
    "Fye": {
        "meaning": "Another way to say something is amazing or cool.",
        "example": "That jacket is fye, where did you get it?"
    },
    "Glow Up": {
        "meaning": "Refers to someone's impressive transformation, often in appearance or confidence.",
        "example": "She had a major glow-up during the holidays!"
    },
    "Yeet": {
        "meaning": "To throw something with enthusiasm or force.",
        "example": "He yeeted his backpack across the room!"
    },
    "W": {
        "meaning": "Stands for \"win,\" used to celebrate success.",
        "example": "You aced the test? That's a W!"
    },
    "L": {
        "meaning": "Short for \"loss,\" often used humorously when something goes wrong.",
        "example": "Dropped my ice cream? That's an L."
    },
    "Vibing": {
        "meaning": "Relaxing or enjoying the moment.",
        "example": "I'm just vibing with some good music."
    },
    "Pog": {
        "meaning": "Used in gaming to express excitement or admiration.",
        "example": "That move was pog, you're a legend!"
    },
    "Glow": {
        "meaning": "Refers to looking fresh, attractive, or confident.",
        "example": "You're glowing today, what's your secret?"
    },
    "Dank": {
        "meaning": "High-quality, especially for memes.",
        "example": "That meme is dank, I'm saving it."
    },
    "Cringe": {
        "meaning": "Something embarrassing or awkward.",
        "example": "Watching my old videos is so cringe."
    },
    "Sus": {
        "meaning": "Short for \"suspicious,\" often used jokingly.",
        "example": "Why is he acting so sus during the game?"
    },
    "Pushing P": {
        "meaning": "Keeping things real or authentic.",
        "example": "They're always honest, they're pushing P."
    },
    "Big Yikes": {
        "meaning": "Used for something extremely awkward or uncomfortable.",
        "example": "He forgot the lyrics on stage? Big yikes!"
    },
    "Flex": {
        "meaning": "To show off, often humorously.",
        "example": "Posting that rare sneaker collection? Major flex."
    },
    "Savage": {
        "meaning": "Someone unapologetically bold or daring.",
        "example": "Her clapback was savage, left everyone speechless."
    },
    "Bet": {
        "meaning": "Used to agree with something or affirm a plan.",
        "example": "You want to go to the mall later? Bet, I'm down."
    },
    "Big Mood": {
        "meaning": "Used to describe something that perfectly expresses how you're feeling.",
        "example": "That's totally a big mood today, I'm exhausted."
    },
    "Clout": {
        "meaning": "Influence or popularity, especially on social media.",
        "example": "She's got mad clout on TikTok, everyone follows her."
    },
    "Shook": {
        "meaning": "To be shocked or surprised, often in a dramatic way.",
        "example": "I was shook when I saw that plot twist in the movie."
    },
    "Fr": {
        "meaning": "Short for \"for real,\" used to emphasize truth.",
        "example": "That was the best concert ever, fr."
    },
    "Vibe Check": {
        "meaning": "A quick judgment or assessment of someone's mood or energy.",
        "example": "Let's do a vibe check, everyone good?"
    },
    "TFW": {
        "meaning": "Stands for \"That feeling when,\" used to start a relatable or funny statement.",
        "example": "TFW you realize you left your phone at home."
    },
    "No Cap": {
        "meaning": "To say something is completely true, no exaggeration.",
        "example": "I'm telling you, this dessert is the best, no cap."
    },
    "Lowkey": {
        "meaning": "Secretly or subtly.",
        "example": "I lowkey want to skip school tomorrow and go to the beach."
    },
    "Highkey": {
        "meaning": "The opposite of lowkey, openly or enthusiastically.",
        "example": "I highkey love that song, it's on repeat."
    },
    "Deadass": {
        "meaning": "Used to say something seriously or with emphasis.",
        "example": "I'm deadass thinking about taking a gap year."
    },
    "Finesse": {
        "meaning": "To skillfully manipulate or handle a situation.",
        "example": "She finessed her way into getting the front-row tickets."
    },
    "Catching Vibes": {
        "meaning": "Enjoying the energy of a situation or feeling the mood.",
        "example": "We were just catching vibes at the park, listening to music."
    },
    "Simp": {
        "meaning": "Refers to someone who is overly attentive or affectionate towards someone they like, often in a way that seems excessive.",
        "example": "He's always buying her gifts, such a simp."
    },
    "Slaps Different": {
        "meaning": "Used to say that something is exceptionally good or unique.",
        "example": "This version of the song slaps different, I love it!"
    },
    "Flexing": {
        "meaning": "Showing off, especially in a flashy or boastful way.",
        "example": "Stop flexing your new car, it's cool, but we get it."
    },
    "Fire": {
        "meaning": "Used to describe something really cool, amazing, or impressive.",
        "example": "Those shoes are fire, where can I get them?"
    },
    "Go Off": {
        "meaning": "Used to encourage someone to speak or act passionately, or to express something strongly.",
        "example": "She went off during the presentation, everyone was impressed!"
    },
    "Mood": {
        "meaning": "A way to relate to someone's current feelings or situation.",
        "example": "That's a whole mood right now, napping sounds perfect."
    },
    "Vibe": {
        "meaning": "Refers to the general feeling or atmosphere of a situation.",
        "example": "The vibe at this party is so chill, I love it."
    },
    "Hype": {
        "meaning": "Excitement or anticipation around something.",
        "example": "The hype for that new movie is real, everyone's talking about it."
    },
    "Tea": {
        "meaning": "Gossip or information, especially the latest news or drama.",
        "example": "What's the tea on that new couple? Are they official?"
    },
    "Tired": {
        "meaning": "Used to express being done with something or feeling fed up.",
        "example": "I'm so tired of hearing about that drama, it's exhausting."
    },
    "Zonked": {
        "meaning": "Completely exhausted or tired.",
        "example": "After that workout, I'm totally zonked."
    },
    "Drippy": {
        "meaning": "Refers to someone's fashion style being fresh and on point.",
        "example": "Look at those sneakers! You're looking drippy today."
    },
    "No Vibe": {
        "meaning": "A situation or person that lacks energy or excitement.",
        "example": "This party has no vibe, it's so quiet."
    },
    "Chuggy": {
        "meaning": "Describes something outdated or trying too hard to be trendy.",
        "example": "That brand is so chuggy, it's not cool anymore."
    },
    "Cracked": {
        "meaning": "Used to describe something that's insanely good, especially in gaming.",
        "example": "He's cracked at Fortnite, no one can beat him."
    },
    "Catch These Hands": {
        "meaning": "A playful way of saying you're ready for a fight or confrontation.",
        "example": "You want to start something? Catch these hands!"
    },
    "Sussy": {
        "meaning": "Short for \"sus,\" used in a similar way to describe someone who is being suspicious or shady.",
        "example": "Why's he acting all sussy around us? What's going on?"
    },
    "Poppin'": {
        "meaning": "Used to describe something that's exciting or very active.",
        "example": "This club is poppin' tonight, let's go!"
    },
    "Vibed Out": {
        "meaning": "Used to say that you're fully enjoying or experiencing a relaxed atmosphere.",
        "example": "We vibed out all day at the beach with no worries."
    },
    "Big Flex": {
        "meaning": "A bold or impressive show of wealth, success, or confidence.",
        "example": "He just pulled up in a brand-new car, big flex!"
    },
    "Woke": {
        "meaning": "Originally meaning socially aware, now it's sometimes used sarcastically to refer to someone overly conscious of social issues.",
        "example": "He's always talking about politics, he's so woke."
    },
    "It's Giving": {
        "meaning": "Used to express a strong impression or vibe about someone or something.",
        "example": "It's giving major 90s fashion, those jeans are so retro!"
    },
    "Steez": {
        "meaning": "Refers to someone's unique style or flair, often in fashion or attitude.",
        "example": "He's got the steez, look at that jacket."
    },
    "Goated": {
        "meaning": "Short for \"Greatest of All Time,\" used to describe something or someone that's exceptional.",
        "example": "That player is goated, best of the season!"
    },
    "Stan": {
        "meaning": "To be an overzealous fan of something or someone.",
        "example": "I stan that artist so hard, she's amazing!"
    },
    "Simping": {
        "meaning": "Acting overly submissive or affectionate towards someone, often at the expense of one's own dignity.",
        "example": "He's been simping hard for her all week, hope it works out."
    },
    "Dead": {
        "meaning": "Used to express something is hilarious, to the point of being so funny it's overwhelming.",
        "example": "That meme had me dead, I couldn't stop laughing."
    }
}

@tool
def get_slang_definitions(input_text: str) -> str:
    """Returns slang terms that are semantically related to the input."""
    result = []
    input_text_lower = input_text.lower()

    for slang, info in slang_dict.items():
        # Naive match: check if any word in the meaning is in input
        if any(word in input_text_lower for word in info["meaning"].lower().split()):
            result.append(f"{slang.upper()}:\nMeaning: {info['meaning']}\nExample: {info['example']}\n")

    # Fallback: if no match, return all (optional)
    if not result:
        for slang, info in slang_dict.items():
            result.append(f"{slang.upper()}:\nMeaning: {info['meaning']}\nExample: {info['example']}\n")

    return "\n".join(result)

**Define the AI Agent state**

In [6]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

**Define the nodes and edges to complete the Agent workflow**

In [7]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_mistral_ai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_mistral_ai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

**Define the system prompt and model**

In [8]:
prompt = """You are a Gen Alpha slang expert. Use the available tools to get the slang for the provided prompt, take a look at the words in the tool more than one if you have too and find the most appropriate gen alpha word and change it to gen alpha slang and give me only the gen alpga slang .

"""

model = init_chat_model("mistral-large-latest", model_provider="mistralai")
abot = Agent(model, [get_slang_definitions], system=prompt)


**Invoke the graph and test it with simple prompt**

In [12]:
messages = [HumanMessage(content="The guy is chaotic, however he is very confident")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'get_slang_definitions', 'args': {'input_text': 'The guy is chaotic, however he is very confident'}, 'id': 'lSQdbAYZX', 'type': 'tool_call'}
Back to the model!


**Output**

In [13]:
print(result['messages'][-1].content)

The guy is skibidi, however he is very sigma


**Conclusion**

*   The AI agent was able to successfully convert the original sentence into gen alpha slang. Congratulations!! now you sound like gen Alpha!





